# RL Taxi Driver 
One of the tutorials recommended on open-ai gym docs. Instructions found here https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [1]:
import gym
from IPython.display import clear_output
from time import sleep
import numpy as np
import random
from IPython.display import clear_output

In [2]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

Radom walk to taxi

In [3]:
env = gym.make("Taxi-v3").env
env.s = 328  # set environment to illustration's state
env.render()

epochs = 0
penalties, reward = 0, 0

frames = []  # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    }
    )

    epochs += 1


print_frames(frames)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 3191
State: 0
Action: 5
Reward: 20
Timesteps taken: 3191
Penalties incurred: 1023


Q Learning

In [4]:
%%time
"""Training the agent"""

q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 47 s, sys: 8.62 s, total: 55.6 s
Wall time: 46.2 s


In [5]:
q_table[328]

array([ -2.40416292,  -2.27325184,  -2.39922046,  -2.35773077,
       -10.52651192, -10.21776502])

In [6]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.41
Average penalties per episode: 0.0


In [10]:
import ray
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "Taxi-v3",
    "render_env": "~/Documents/ExploringRL/Recordings",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 4,
    "horizon": 10000,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    # Since learning is most of the time done on the local worker,
    # it may help to provide one or more GPUs to that worker via the num_gpus setting
    "num_gpus": 1,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
}

# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for _ in range(10):
    print(trainer.train())

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()

In [26]:
import pickle
with open("trainer.pkl", 'wb') as f:
    pickle.dump(agent, f)

In [ ]:
"""Evaluate PPO agent's performance after learning"""

total_epochs, total_penalties = 0, 0
episodes = 100
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = trainer.compute_single_action(state)
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

In [ ]:
env = gym.make("Taxi-v3").env
print(env.s)
env.render()

In [ ]:
env.reset()

In [6]:
import pandas as pd
import json
import os
import shutil
import sys
import ray
import ray.rllib.agents.ppo as ppo


In [1]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger

In [ ]:
info = ray.init(ignore_reinit_error=True)

In [8]:
checkpoint_root = "tmp/ppo/taxi"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [9]:
SELECT_ENV = "Taxi-v3"
N_ITER = 20

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config["framework"] = "torch"
config["num_gpus"] = 4
# config["callbacks"] = WandbLoggerCallback(api_key_file="/home/sem22h2/.netrc", project="taxi-v3")

agent = ppo.PPOTrainer(config, env=SELECT_ENV)
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

ValueError: `callbacks` must be a callable method that returns a subclass of DefaultCallbacks, got <ray.tune.integration.wandb.WandbLoggerCallback object at 0x7fc5c6d16d30>!

In [21]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())


[Parameter containing:
tensor([[ 0.0484,  0.0386, -0.0780,  ..., -0.0539, -0.0186, -0.0189],
        [-0.0605, -0.0826, -0.0100,  ..., -0.0543, -0.0494, -0.0333],
        [ 0.0751,  0.0094,  0.0455,  ...,  0.1059,  0.0092, -0.0261],
        [-0.0117, -0.0598,  0.0818,  ...,  0.0364, -0.0492, -0.0255],
        [-0.0592,  0.0431, -0.0732,  ..., -0.0676,  0.0676,  0.0304],
        [ 0.0083,  0.0732,  0.0018,  ...,  0.0290,  0.0547,  0.0975]],
       device='cuda:0', requires_grad=True),
 Parameter containing:
tensor([ 0.0043,  0.0148,  0.0079,  0.0059, -0.0152, -0.0172], device='cuda:0',
       requires_grad=True),
 Parameter containing:
tensor([[ 0.0708,  0.0101, -0.0280,  ..., -0.0074, -0.0139, -0.0591],
        [ 0.0276,  0.0846, -0.0561,  ...,  0.0012, -0.0517,  0.1160],
        [-0.0172,  0.0208, -0.0893,  ..., -0.0526, -0.0121, -0.0697],
        ...,
        [-0.0116, -0.0449, -0.0796,  ..., -0.0394,  0.0614,  0.0321],
        [ 0.0129, -0.0071,  0.0489,  ...,  0.0340, -0.0889,  0.0

AttributeError: 'FullyConnectedNetwork' object has no attribute 'base_model'

In [1]:
import ray
from ray.tune.integration.wandb import WandbLoggerCallback

In [17]:
SELECT_ENV = "Taxi-v3"
N_ITER = 20


ray.init()
ray.tune.run(
    "PPO",
    stop={"training_iteration": 15},
    config={
        "env": SELECT_ENV,
        "record_env": True,
        "framework": "torch",
        "num_cpus_per_worker": 2,
        "num_gpus": 2,
        "num_workers": 4,
    },
    local_dir="logs",
    callbacks=[
            WandbLoggerCallback(api_key="c36c598399c6c7f2f0b446aac164da6c7956a263", project="Taxi_v3")],
)


wandb: Currently logged in as: danky. Use `wandb login --relogin` to force relogin


(PPOTrainer pid=39908) 2022-07-08 07:50:26,739	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=39908) 2022-07-08 07:50:26,739	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=40009) Setting the path for recording to /home/sem22h2/Documents/ExploringRL/logs/PPO/PPO_Taxi-v3_d9332_00000_0_2022-07-08_07-50-19/
(RolloutWorker pid=40010) Setting the path for recording to /home/sem22h2/Documents/ExploringRL/logs/PPO/PPO_Taxi-v3_d9332_00000_0_2022-07-08_07-50-19/
(RolloutWorker pid=40008) Setting the path for recording to /home/sem22h2/Documents/ExploringRL/logs/PPO/PPO_Taxi-v3_d9332_00000_0_2022-07-08_07-50-19/
(RolloutWorker pid=40007) Setting the path for recording to /home/sem22h2/Documents/ExploringRL/logs/PPO/PPO_Taxi-v3_d9332_00000_0_2022-07-08_07-50-19/


Trial name,status,loc
PPO_Taxi-v3_d9332_00000,RUNNING,129.132.4.155:39908


(PPOTrainer pid=39908) 2022-07-08 07:50:34,834	WARNING util.py:65 -- Install gputil for GPU system monitoring.


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_Taxi-v3_d9332_00000,RUNNING,129.132.4.155:39908,14,268.241,56000,-435.8,-7,-812,177.14


Result for PPO_Taxi-v3_d9332_00000:
  agent_timesteps_total: 60000
  counters:
    num_agent_steps_sampled: 60000
    num_agent_steps_trained: 60000
    num_env_steps_sampled: 60000
    num_env_steps_trained: 60000
  custom_metrics: {}
  date: 2022-07-08_07-55-24
  done: true
  episode_len_mean: 177.92
  episode_media: {}
  episode_reward_max: -7.0
  episode_reward_mean: -404.75
  episode_reward_min: -776.0
  episodes_this_iter: 21
  episodes_total: 320
  experiment_id: 7b251ced4d844a0fb8570d8f93aed46f
  hostname: sassauna2.ee.ethz.ch
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.3
          cur_lr: 5.0000000000000016e-05
          entropy: 1.2126518983353851
          entropy_coeff: 0.0
          kl: 0.01616111890676137
          policy_loss: -0.040561869820599915
          total_loss: 9.644420728375835
          vf_explained_var: -0.04814397872135203
          vf_loss: 9.680134253348074
        model: {}
    num

agent_timesteps_total,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
counters/num_agent_steps_sampled,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
counters/num_agent_steps_trained,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
counters/num_env_steps_sampled,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
counters/num_env_steps_trained,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
episode_len_mean,▆██▇▆▇▆▆▆▅▃▁▂▁▂
episode_reward_max,▁▁▁▁████▄▄█████
episode_reward_mean,▂▁▁▁▂▂▃▃▄▄▅▆▆▇█
episode_reward_min,▄▁▁▁▁▁▂▂▂▂▅▇▇▇█
episodes_this_iter,▁▁▁▁▅▁▂▁▄▂▇█▁▅▂
episodes_total,▁▁▂▂▃▃▄▄▅▅▆▇▇██


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_Taxi-v3_d9332_00000,TERMINATED,129.132.4.155:39908,15,288.313,60000,-404.75,-7,-776,177.92


(RolloutWorker pid=40009) E0708 07:55:28.858038367   40113 chttp2_transport.cc:1111]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(RolloutWorker pid=40008) E0708 07:55:28.856797478   40093 chttp2_transport.cc:1111]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
2022-07-08 07:55:28,966	INFO tune.py:747 -- Total run time: 310.04 seconds (309.48 seconds for the tuning loop).


In [ ]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean")

# or simply get the last checkpoint (with highest "training_iteration")
last_checkpoint = analysis.get_last_checkpoint()
# if there are multiple trials, select a specific trial or automatically
# choose the best one according to a given metric
last_checkpoint = analysis.get_last_checkpoint(
    metric="episode_reward_mean", mode="max"
)

In [16]:
ray.shutdown()

In [ ]:
Creaete NAS Gym env
